# EDA jugadores de Fifa
En este informe se busca analizar... 
### (Escribir proposito del informe)
puede ser algo tipo identificar las variables clave para predecir el precio de un jugador y observar el efecto de estas variables sobre el precio.


## Analisis preliminar de base de datos
Primero, importo las librerías a utilizar en el análisis.

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns                       #visualisation
import matplotlib.pyplot as plt             #visualisation
%matplotlib inline     
sns.set(color_codes=True)

Luego, descargo la base de datos y obtengo una primera imagen sobre que contiene

In [3]:
import pandas as pd


df = pd.read_csv("./footballData.csv")
nombres = df['long_name']

print(df.shape)
df.head(4).transpose()

(18944, 106)


,0,1,2,3
sofifa_id,158023,20801,200389,188545
player_url,https://sofifa.com/player/158023/lionel-messi/...,https://sofifa.com/player/20801/c-ronaldo-dos-...,https://sofifa.com/player/200389/jan-oblak/210002,https://sofifa.com/player/188545/robert-lewand...
short_name,L. Messi,Cristiano Ronaldo,J. Oblak,R. Lewandowski
long_name,Lionel Andrés Messi Cuccittini,Cristiano Ronaldo dos Santos Aveiro,Jan Oblak,Robert Lewandowski
age,33,35,27,31
...,...,...,...,...
lb,62+3,61+3,32+3,61+3
lcb,52+3,54+3,33+3,60+3
cb,52+3,54+3,33+3,60+3
rcb,52+3,54+3,33+3,60+3


En el análisis preliminar se observa que la data contiene 106 columnas y 18944 filas.
Por ende, primero se debe identificar que columnas son claves para el análisis y descartar las demás.

In [4]:
df.dtypes

sofifa_id      int64
player_url    object
short_name    object
long_name     object
age            int64
               ...  
lb            object
lcb           object
cb            object
rcb           object
rb            object
Length: 106, dtype: object

In [5]:
df.loc[:,df.isnull().any()].isnull().sum()

club_name                 225
league_name               225
league_rank               225
release_clause_eur        995
player_tags             17536
team_position             225
team_jersey_number        225
loaned_from             18186
joined                    983
contract_valid_until      225
nation_position         17817
nation_jersey_number    17817
pace                     2083
shooting                 2083
passing                  2083
dribbling                2083
defending                2083
physic                   2083
gk_diving               16861
gk_handling             16861
gk_kicking              16861
gk_reflexes             16861
gk_speed                16861
gk_positioning          16861
player_traits           10629
defending_marking       18944
dtype: int64

Al observar las columnas con valores faltantes, y cuantos les faltan, se observa cierta relación entre estas.
Se observan muchos valores faltantes para los traits de arquero, por ende, se decide descartar los arqueros y las columnas con su respectiva información del modelo. Esto es porque se considera que los arqueros requerirían un análisis por separado.

#### Analizo las columnas short_name, long_name y sofifa_id

In [6]:
df.loc[df['short_name'].duplicated(), 'short_name'].count()

1060

In [7]:
df.loc[df['long_name'].duplicated(), 'long_name'].count()

48

In [8]:
df.loc[df['sofifa_id'].duplicated(), 'sofifa_id'].count()

0

Cómo short_name y long_name contienen valores duplicados y sofifa_id no apora inguna información, se decide eliminar las tres columnas y usar el índice para diferenciar a los jugadores.

#### (Es horrible trabajar sin el nombre de los jugadores, pero no hay  ninguna forma linda de justificar trbaajar con long_name)



#### Analizo columna players_positions

In [9]:
type(df['player_positions'][0])

str

La posicion de los jugadores está en formato String, por lo que es conveniente pasarlo a una lista que contenga las posibles posiciones de cada jugador por separado.

In [10]:
df['player_positions']  = [item.split(', ') for item in df['player_positions']]
df['player_positions'].head()

0    [RW, ST, CF]
1        [ST, LW]
2            [GK]
3            [ST]
4       [LW, CAM]
Name: player_positions, dtype: object

### Selecciono columnas para el análisis
Entonces, procedemos a descartar las columnas que carecen de información importante (como player_url, face_id, etc), las que contienen información similar a la de otra columna y que no aporta al análisis (cómo short_name y sofifa_id cuya información se encuentra en long_name, o dob, que informa la fecha de nacimiento del jugador, y mantenemos la edad en vez), en cuánto a atributos de jugador, mantenemos las columnas principales (por ejemplo descartamos attacking_finishing pero mantenemos shooting) ya que buscamos un análisis genérico de los jugadores y consideramos que esos atributos se requerirían para un análisis más puntual por posición. Finalmente, descartamos las columnas ls, st, etc, ya que poseen información sobre como se adapta el jugador para jugar en otras posiciones, lo cuál consideramos reduntante.

Se seleccionan las columnas a utilizar en el analisis:


*   age
*   height_cm
*   weight_kg
*   nationality
*   club_name
*   league_name
*   league_rank
*   team_jersey_number
*   nation_jersey_number
*   overall
*   value_eur
*   wage_eur
*   player_positions
*   preferred_foot
*   skill_moves
*   team_jersey_number
*   pace
*   shooting
*   passing
*   dribbling
*   defending
*   physic

Por ende, en primera instancia se mantienen 22 columnas de las 106 iniciales y se eliminan los jugadores cuya posición sea GK (arquero)

In [11]:
columnas = ['long_name', 'value_eur', 'wage_eur','age', 'height_cm', 'weight_kg', 
            'nationality', 'club_name', 'league_name', 'league_rank', 
            'team_jersey_number', 'nation_jersey_number', 'overall', 
            'player_positions', 'preferred_foot', 'skill_moves', 
            'pace', 'shooting', 'passing', 'dribbling', 'defending', 'physic']

df = df.loc[['GK' not in item for item in df['player_positions']], columnas]

print(df.shape)
df.head().transpose()

(16860, 22)


,0,1,3,4,5
long_name,Lionel Andrés Messi Cuccittini,Cristiano Ronaldo dos Santos Aveiro,Robert Lewandowski,Neymar da Silva Santos Júnior,Kevin De Bruyne
value_eur,67500000,46000000,80000000,90000000,87000000
wage_eur,560000,220000,240000,270000,370000
age,33,35,31,28,29
height_cm,170,187,184,175,181
weight_kg,72,83,80,68,70
nationality,Argentina,Portugal,Poland,Brazil,Belgium
club_name,FC Barcelona,Juventus,FC Bayern München,Paris Saint-Germain,Manchester City
league_name,Spain Primera Division,Italian Serie A,German 1. Bundesliga,French Ligue 1,English Premier League
league_rank,1.0,1.0,1.0,1.0,1.0


## Plan de Análisis
Establecidas las columnas a utilizar en el análisis, se procede a establecer un plan de análisis. Primero se realizará...

plt.figure(figsize=(10,5))
c = df.corr()
sns.heatmap(c,cmap="BrBG",annot=True)


In [40]:
df_avg_val_numero_camiseta = df.groupby(['team_jersey_number']).agg({'value_eur':'mean', 'team_jersey_number':'count'}).sort_values(['value_eur'],ascending=False)
df_avg_val_numero_camiseta.columns = ['mean_value', 'count']
sns.barplot(x=df_avg_val_numero_camiseta.head(10).index, y = df_avg_val_numero_camiseta['mean_value'].head(10))
sns.set(rc={'figure.figsize':(32,8.27)})

ValueError: 'team_jersey_number' is both an index level and a column label, which is ambiguous.

In [38]:
df_count_numero_camiseta = df.pivot_table(values='value_eur', index="team_jersey_number", aggfunc=[np.mean, 'count'])
df_avg_val_numero_camiseta


,mean_value,count
team_jersey_number,,
59.0,6.315000e+06,5
95.0,4.712500e+06,8
10.0,4.695891e+06,589
7.0,3.692131e+06,603
81.0,3.625000e+06,7
66.0,3.610833e+06,24
91.0,3.552857e+06,14
9.0,3.497543e+06,584
98.0,3.378800e+06,25
